In [1]:
using CUDA
using Ipaper
using BenchmarkTools

In [49]:
function time_average_kernel(data, result)
    idx = threadIdx().x + (blockIdx().x - 1) * blockDim().x
    idy = threadIdx().y + (blockIdx().y - 1) * blockDim().y
    if idx <= size(data, 1) && idy <= size(data, 2)
        sum = 0.0f0
        for t in 1:size(data, 3)
            sum += data[idx, idy, t]
        end
        result[idx, idy] = sum / size(data, 3)
    end
    nothing
end

function zonalmean_gpu(data)
    _size = size(data)[1:2]
    result = CUDA.zeros(Float32, _size)
    
    threads_per_block = (32, 32)
    blocks_per_grid = (ceil(Int, 1000 / 32), ceil(Int, 1000 / 32))
    
    @cuda blocks=blocks_per_grid threads=threads_per_block time_average_kernel(data, result)    
    result
end

function zonalmean_cpu(data::AbstractArray{<:Real,3})
    nlon, nlat, ntime = size(data)
    result = zeros(Float32, nlon, nlat)
    
    # j在前，速度翻10倍
    @inbounds for j=1:nlat, i in 1:nlon
        sum = 0.0f0
        for t in 1:ntime
            sum += data[i, j, t]
        end
        result[i, j] = sum / ntime
    end
    result
end

zonalmean_cpu (generic function with 1 method)

In [7]:
# 假设数据类型为 Float32，创建一个随机数据数组
data = CUDA.rand(Float32, 1000, 1000, 30)
x = Array(data)
obj_size(x)


Array{Float32, 3} | (1000, 1000, 30) | 114.44 Mb


In [5]:
obj_size((10000, 10000, 30), Float32) # 11Gb

Float32 | (10000, 10000, 30) | 11444.09 Mb


In [40]:
@time r1 = zonalmean_gpu(data);
@time r2 = zonalmean_cpu(x);
Array(r1) == r2

  0.000132 seconds (21 allocations: 752 bytes)
  0.017931 seconds (2 allocations: 3.815 MiB)


true

In [45]:
@btime r1 = zonalmean_gpu($data);
@btime r2 = zonalmean_cpu($x);

  11.100 μs (21 allocations: 752 bytes)
  17.450 ms (2 allocations: 3.81 MiB)


In [46]:
@btime r2 = zonalmean_cpu($x);

  17.504 ms (2 allocations: 3.81 MiB)


In [48]:
17.45*1000/11.100

1572.0720720720722